# Which vectorization?

In [1]:
!pip install mlflow boto3 awscli

  Using cached botocore-1.35.99-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.10.4-py3-none-any.whl.metadata (1.7 kB)
INFO: pip is looking at multiple versions of awscli to determine which version is compatible with other requirements. This could take a while.
  Using cached awscli-1.44.4-py3-none-any.whl.metadata (11 kB)
INFO: pip is still looking at multiple versions of awscli to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 8.2 MB/s  0:00:01m0:00:0100:01
Using cached s3transfer-0.10.4-py3-none-any.whl (83 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 11.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !aws configure
import os

os.environ['AWS_ACCESS_KEY_ID'] = 'acesskeyid123'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'secretaccess'
os.environ['AWS_DEFAULT_REGION'] = 'eu-west-3' # or your preferred region

In [3]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://localhost:5000")

In [4]:
# Set or create an experiment
mlflow.set_experiment("Exp 2 - BoW vs TfIdf")

2025/12/29 21:39:06 INFO mlflow.tracking.fluent: Experiment with name 'Exp 2 - BoW vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-manel/artifacts/3', creation_time=1767040746310, experiment_id='3', last_update_time=1767040746310, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf', tags={}>

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [6]:
# df = pd.read_csv('/content/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df = pd.read_csv("/Users/manel/Library/Mobile Documents/com~apple~CloudDocs/Github/Manel_Personal_Projects/Projet_MLOPS/reddit_preprocessing.csv").dropna(subset=["clean_comment"])
df.shape

(36662, 2)

In [7]:

# Step 1: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # Step 2: Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Step 6: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")


2025/12/29 21:39:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/12/29 21:39:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run BoW_(1, 1)_RandomForest at: http://localhost:5000/#/experiments/3/runs/2cb4c37967044379845c4d8272466383.
2025/12/29 21:39:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2025/12/29 21:39:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/12/29 21:39:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run TF-IDF_(1, 1)_RandomForest at: http://localhost:5000/#/experiments/3/runs/56f4ad0dcee74ea7ac2008cbaafc6bb8.
2025/12/29 21:39:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/

## unigram, bigram, trigram for BOW
## unigram, bigram, trigram for TF-IDF
## as a conclusion, we have to use Trigrams with TF-IDF

code optimisé: BoW vs TF-IDF Experiments

In [ ]:
# ===============================
# Notebook :BoW vs TF-IDF Experiments
# ===============================

# ------------------------------
# Install required packages
# ------------------------------
!pip install mlflow boto3 awscli --quiet

# ------------------------------
# Imports
# ------------------------------
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn

# ------------------------------
# AWS (if needed)
# ------------------------------
os.environ['AWS_ACCESS_KEY_ID'] = 'acesskeyid123'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'secretaccess'
os.environ['AWS_DEFAULT_REGION'] = 'eu-west-3'  # adapt your region

# ------------------------------
# MLflow setup
# ------------------------------
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Exp 2 - BoW vs TF-IDF")

# ------------------------------
# Load preprocessed dataset
# ------------------------------
df = pd.read_csv("/Users/manel/Library/Mobile Documents/com~apple~CloudDocs/Github/Manel_Personal_Projects/Projet_MLOPS/reddit_preprocessing.csv").dropna(subset=["clean_comment"])
print("Dataset shape:", df.shape)
print(df['category'].value_counts())

# ------------------------------
# Function to run a single experiment
# ------------------------------
def run_experiment(vectorizer_type, ngram_range, max_features, vectorizer_name):
    """
    Train Random Forest on given vectorizer type and ngram_range.
    Logs everything in MLflow: params, metrics, model, confusion matrix.
    """
    # Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
    )
    
    # Fit transform
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    # MLflow logging
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")
        mlflow.set_tag("dataset", "Reddit Comments")
        mlflow.set_tag("feature_extraction", vectorizer_name)
        mlflow.set_tag("notes", f"RandomForest with {vectorizer_name}, ngram={ngram_range}, max_features={max_features}")

        # Log params
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)
        
        # Random Forest params
        n_estimators = 200
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        
        # Train model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)
        
        # Predictions
        y_pred = model.predict(X_test)
        
        # Log metrics
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in report.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)
        
        # Confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8,6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name} {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()
        
        # Log model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")
        
        print(f"✅ Done: {vectorizer_name} {ngram_range} | Accuracy: {accuracy:.4f}")

# ------------------------------
# Run experiments
# ------------------------------
ngram_ranges = [(1,1), (1,2), (1,3)]
max_features = 5000

for ngram_range in ngram_ranges:
    # BoW
    run_experiment("BoW", ngram_range, max_features, "BoW")
    # TF-IDF
    run_experiment("TF-IDF", ngram_range, max_features, "TF-IDF")

print("All experiments completed! ✅")
